In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from Imports import *
import seaborn as sns
from scipy import stats

ModuleNotFoundError: No module named 'Imports'

### Target values (%MGS, binary)

In [2]:
# Load target data
benMGS = pickle.load(open('../../Datasets/BenV2/BenV2MGS.pkl','rb'))
benMGS.columns = ['MGS']
benMGS = binarize(benMGS,'MGS',60)

NameError: name 'pickle' is not defined

In [ ]:
# Remove entries with missing descriptor data
missing = ['i005','i006','i014','i015','i027','i028','i033','i083','i086','i059','i132','i136','i138','i137','i157','i158','i159','i183','i184','i185','i186','i187','i188','i189','i190','i191','i192','i194','i233','i234']
for row in benMGS.itertuples():
    if row.Index in missing:
        benMGS.drop([row.Index],inplace=True)

In [5]:
labels = benMGS['MGS'].values.reshape(-1,1) # For regression
labelsClf = benMGS['Label'].values.reshape(-1,1) # For classification

NameError: name 'benMGS' is not defined

In [ ]:
ax = sns.countplot(x="Label", data=benMGS, palette="Set3")
ax.set(xlabel='Active [1] or not [0]', ylabel='N. of counts')

### 3D structures

In [ ]:
import os.path

In [ ]:
def read_xyz(filename): # filename -> full path!
    global nat, sym, pos
    f = open(filename, "r")
    nat = int(f.readline())
    sym=[]
    pos=np.zeros((nat,3))
    f.readline()
    for i in range (0,nat):
        line = f.readline()
        line_s = line.split()
        sym.append(line_s[0])
        pos[[i],[0]]=float(line_s[1]); pos[[i],[1]]=float(line_s[2]); pos[[i],[2]]=float(line_s[3])
    return(nat,sym,pos)

In [ ]:
xyz={}
counter=0
for i in benMGS.index:
    # filename="../../Datasets/BenV2/xyzFiles/OBabel/"+i+".xyz" # converted from oBabel
    filename="../../Datasets/BenV2/xyzFiles/DFT/"+i+".xyz" # DFT optimised
    read_xyz(filename)
    xyz[counter]=i,benMGS.MGS[i],benMGS.Label[i],nat,sym,pos
    counter=counter+1

Some info...
* 0 -> name, e.g i001
* 1 -> MGS
* 2 -> Label [0] non active, [1] active
* 3 -> N. of atoms in the molecule
* 4 -> Atomic symbols. e.g. xyz[126][4][3] gets the atomic symbol of the fourth atom in molecule indexed 126
* 5 -> Atomic positions. e.g. xyz[126][5][0][2] gets the z-coordinate of the first atom in molecule indexed 126

### Cliques

In [ ]:
# Read in the smiles
smiles = pd.read_csv('../../Datasets/BenV2/BenV2Smi.csv')

In [ ]:
benMGS_sm=benMGS
benMGS_sm['ID'] = benMGS.index

In [ ]:
cliques = pickle.load(open('../../Datasets/BenV2/BenV2Clq.pkl','rb'))

In [ ]:
cliques['ID'] = cliques.index

In [ ]:
df_cliques = pd.merge(cliques, benMGS_sm, on='ID')

In [ ]:
df_cliques.drop(['ID', 'MGS', 'Label'], axis = 1, inplace=True)
clq = np.array(df_cliques)

### Standard Descriptors

In [ ]:
df_smiles = pd.merge(smiles, benMGS_sm, on='ID')

In [ ]:
# Add H's and create mol objects from each SMILE
from rdkit import Chem

mol_h=[]

for i in df_smiles['SMILES']:
    mol=Chem.MolFromSmiles(i)
    mol_h.append(Chem.AddHs(mol))

In [ ]:
from rdkit.Chem import Descriptors

SDTR=[]
for m in mol_h:
    desc=[]
    desc.append(Chem.Descriptors.ExactMolWt(m))
    desc.append(Chem.rdMolDescriptors.CalcNumLipinskiHBA(m))
    desc.append(Chem.rdMolDescriptors.CalcNumLipinskiHBD(m))
    desc.append(Chem.rdMolDescriptors.CalcNumRotatableBonds(m))
    desc.append(Chem.rdMolDescriptors.CalcNumHBD(m))
    desc.append(Chem.rdMolDescriptors.CalcNumHBA(m))
    desc.append(Chem.rdMolDescriptors.CalcNumHeteroatoms(m))
    desc.append(Chem.rdMolDescriptors.CalcNumAmideBonds(m))
    desc.append(Chem.rdMolDescriptors.CalcFractionCSP3(m))
    desc.append(Chem.rdMolDescriptors.CalcNumRings(m))
    desc.append(Chem.rdMolDescriptors.CalcNumAromaticRings(m))
    desc.append(Chem.rdMolDescriptors.CalcNumAliphaticRings(m))
    desc.append(Chem.rdMolDescriptors.CalcNumSaturatedRings(m))
    desc.append(Chem.rdMolDescriptors.CalcNumHeterocycles(m))
    desc.append(Chem.rdMolDescriptors.CalcNumAromaticHeterocycles(m))
    desc.append(Chem.rdMolDescriptors.CalcNumSaturatedHeterocycles(m))
    desc.append(Chem.rdMolDescriptors.CalcNumAliphaticHeterocycles(m))
    desc.append(Chem.rdMolDescriptors.CalcNumSpiroAtoms(m))
    SDTR.append(desc)
SDTR_n=np.array(SDTR)

### SFs

In [ ]:
sfs = pickle.load(open('../../Datasets/BenV2/BenV2SymObabelOptRF.pkl','rb'))
sfs['ID'] = sfs.index

In [ ]:
df_sfs = pd.merge(sfs, benMGS_sm, on='ID')

In [ ]:
df_sfs.drop(['ID', 'MGS', 'Label'], axis = 1, inplace=True)

In [ ]:
sf = np.array(df_sfs)

### SOAPs

In [ ]:
import quippy

In [ ]:
from quippy import descriptors
import ase
import ase.build
from ase import Atoms
from ase.visualize import view

In [ ]:
# SOAP descriptor, Carbon, Oxygen and Hydrogen both taken as SOAP centers.
# Basic line straight out from the tutorial
desc = descriptors.Descriptor("soap cutoff=3 l_max=4 n_max=4 atom_sigma=0.5 n_Z=3 Z={1 6 8} n_species=3 species_Z={1 6 8}")
# average -> average over the densities, and THEN the rotational invariant.

In [ ]:
soaps={}
separator=''
for i in xyz:
    at_sym = str(separator.join(xyz[i][4]))
    at_pos = np.array(xyz[i][5])
    at = Atoms(at_sym, positions=at_pos)
    #print(i,desc.sizes(at),desc.n_dim)
    soaps[i]=desc.calc(at)

In [ ]:
# A rather silly descriptor: the average of each SOAP vector / molecule

ave_soap=np.zeros(len(soaps[0]['data'][0])) # always the same dimensionality for each molecule
silly_soap_D=np.zeros((len(soaps),len(soaps[0]['data'][0])))

for j in range (0,len(soaps)):
    n_soaps = len(soaps[j]['data'])
    for i in range (0,n_soaps):
        ave_soap=ave_soap+soaps[j]['data'][i]
    ave_soap=ave_soap/float(i)
    silly_soap_D[j]=ave_soap

In [ ]:
# Target values...
targets=np.zeros(len(soaps))
for j in range (0,len(soaps)):
    targets[j] = xyz[j][2]

### HBonds (2)

In [ ]:
hb = smiles = pd.read_csv('../../Datasets/BenV2/BenV2Hbonds2.csv')

In [ ]:
df_hb = pd.merge(hb, benMGS_sm, on='ID')

In [ ]:
df_hb.drop(['ID', 'MGS','Label'], axis = 1, inplace=True)
# hbs = np.array(df_hb.drop).ravel()

In [ ]:
hbs=np.array(df_hb['Hbonds'])

### Stratified K fold over the *whole* dataset

Same splits across the different descriptors (ensured by a given value of ```random state```). ```n_splits``` determines the training/test sets ratio. Controversial choice, I guess, but with only 200 datapoints we can't play the training/validation/test game...

In [ ]:
skf = StratifiedKFold(n_splits=20, shuffle=True, random_state=7654)

In [ ]:
# Cliques
Xt_clq=[] ; Xte_clq=[] ; yt=[] ; yte=[]; idx_t=[]; idx_te=[]

for train_index, test_index in skf.split(clq,targets.ravel()):
    Xt_clq.append(clq[train_index])
    Xte_clq.append(clq[test_index])
    yt.append(targets.ravel()[train_index])
    yte.append(targets.ravel()[test_index])
    idx_t.append(train_index)
    idx_te.append(test_index)

In [ ]:
# Standard descriptors
Xt_sds=[] ; Xte_sds=[]

for train_index, test_index in skf.split(SDTR_n,targets.ravel()):
    Xt_sds.append(SDTR_n[train_index])
    Xte_sds.append(SDTR_n[test_index])

In [ ]:
# Symmetry functions 
Xt_sfs=[] ; Xte_sfs=[]

for train_index, test_index in skf.split(sf,targets.ravel()):
    Xt_sfs.append(sf[train_index])
    Xte_sfs.append(sf[test_index])

In [ ]:
# SOAPs 
Xt_sop=[] ; Xte_sop=[]

for train_index, test_index in skf.split(silly_soap_D,targets.ravel()):
    Xt_sop.append(silly_soap_D[train_index])
    Xte_sop.append(silly_soap_D[test_index])

In [ ]:
# HBs 
Xt_hb=[] ; Xte_hb=[]

for train_index, test_index in skf.split(hbs,targets.ravel()):
    Xt_hb.append(hbs[train_index])
    Xte_hb.append(hbs[test_index])

### Fit each model n_splits times

In [ ]:
# Same model across the different splits and features
params = {'max_features': 'log2', 'min_samples_leaf':15, 'n_estimators': 1000}
clf=RandomForestClassifier(criterion='entropy',n_estimators=params['n_estimators'],max_features=params['max_features'],min_samples_leaf=params['min_samples_leaf'])

In [ ]:
# Cliques
prd_clq=[] ; exp=[] ; idx=[]
for i,j,k,l,m in zip(Xt_clq,yt,Xte_clq,yte,idx_te):
    clf.fit(i,j);
    prd_clq.append(clf.predict(k))
    exp.append(l)
    idx.append(m)
prd_clq_f = np.array(list(np.concatenate(prd_clq).flat))
exp_f = np.array(list(np.concatenate(exp).flat))
idx_f = np.array(list(np.concatenate(idx).flat))
matthews_corrcoef(prd_clq_f,exp_f)

In [ ]:
cm_clq = confusion_matrix(prd_clq_f,exp_f)
plot_conf_mat(cm_clq,["Inactive","Active"],normalise=True,percent=True,label=True,stat=False,name="exampleCM.png")

In [ ]:
# Standard descriptors
prd_sds=[]
for i,j,k in zip(Xt_sds,yt,Xte_sds):
    clf.fit(i,j);
    prd_sds.append(clf.predict(k))
prd_sds_f = np.array(list(np.concatenate(prd_sds).flat))
matthews_corrcoef(prd_sds_f,exp_f)

In [ ]:
cm_sds = confusion_matrix(prd_sds_f,exp_f)
plot_conf_mat(cm_sds,["Inactive","Active"],normalise=True,percent=True,label=True,stat=False,name="exampleCM.png")

In [ ]:
# Symmetry functions
prd_sfs=[]
for i,j,k in zip(Xt_sfs,yt,Xte_sfs):
    clf.fit(i,j);
    prd_sfs.append(clf.predict(k))
prd_sfs_f = np.array(list(np.concatenate(prd_sfs).flat))
matthews_corrcoef(prd_sfs_f,exp_f)

In [ ]:
cm_sfs = confusion_matrix(prd_sfs_f,exp_f)
plot_conf_mat(cm_sfs,["Inactive","Active"],normalise=True,percent=True,label=True,stat=False,name="exampleCM.png")

In [ ]:
# SOAPs
prd_sop=[]
for i,j,k in zip(Xt_sop,yt,Xte_sop):
    clf.fit(i,j);
    prd_sop.append(clf.predict(k))
prd_sop_f = np.array(list(np.concatenate(prd_sop).flat))
matthews_corrcoef(prd_sop_f,exp_f)

In [ ]:
cm_sop = confusion_matrix(prd_sop_f,exp_f)
plot_conf_mat(cm_sop,["Inactive","Active"],normalise=True,percent=True,label=True,stat=False,name="exampleCM.png")

In [ ]:
# HBonds
prd_hb=[]
for i,j,k in zip(Xt_hb,yt,Xte_hb):
    clf.fit(i.reshape(-1,1),j);
    prd_hb.append(clf.predict(k.reshape(-1,1)))
prd_hb_f = np.array(list(np.concatenate(prd_hb).flat))
matthews_corrcoef(prd_hb_f,exp_f)

In [ ]:
cm_hb = confusion_matrix(prd_hb_f,exp_f)
plot_conf_mat(cm_hb,["Inactive","Active"],normalise=True,percent=True,label=True,stat=False,name="exampleCM.png")

### Max voting

In [ ]:
final_pred = np.array([])
votes = []

for j,k,l in zip(prd_sds_f,prd_sfs_f,prd_sop_f):
    max_vote=stats.mode(np.array((j,k,l)))
    votes.append(np.asscalar(max_vote[1]))
    final_pred = np.append(final_pred, max_vote[0])
matthews_corrcoef(final_pred,exp_f)

In [ ]:
cm_all = confusion_matrix(final_pred,exp_f)
plot_conf_mat(cm_all,["Inactive","Active"],normalise=True,percent=True,label=True,stat=False,name="exampleCM.png")

In [ ]:
from matplotlib.colors import LinearSegmentedColormap
df_all = pd.DataFrame({'sds':prd_sds_f, 'sfs':prd_sfs_f, 'sop':prd_sop_f, 'all':final_pred, 'exp':exp_f})
colors = ('paleturquoise', 'deeppink')
fig, ax = plt.subplots(figsize=(10,40))
cmap = LinearSegmentedColormap.from_list('Custom', colors, len(colors))
ax = sns.heatmap(df_all, cmap=cmap, linewidths=0.1, linecolor='gray')
    
# Set the colorbar labels
colorbar = ax.collections[0].colorbar
colorbar.set_ticks([0.25,0.75])
colorbar.set_ticklabels(['Inactive', 'Active'])

In [ ]:
tru=[]
for i,j in zip(final_pred,exp_f):
    if (i == j):
        tru.append(1) # True
    else:
        tru.append(0) # False

In [ ]:
df_all_v=pd.DataFrame({'truth':tru, 'votes':votes})

sure_true=0
crazy_false=0

for i,j in zip(df_all_v['truth'],df_all_v['votes']):
    if (i == 1) and (j == 3):
        sure_true=sure_true+1
    if (i == 0) and (j == 3):
        crazy_false=crazy_false+1    

print("Correct predictions, 3/3 votes:", sure_true)
print("Incorrect predictions, 3/3 votes:", crazy_false)